# Compute i coefficients

In [1]:
import brightway2 as bw
import pandas as pd
import os
import numpy as np

# Set working directry
path = "."
os.chdir(path)

# Local functions
from utils.lookup_func import lookup_geothermal

# Set project
bw.projects.set_current("Geothermal")

# Retrieve activities
wellhead, diesel, steel, cement, water, \
drilling_mud, drill_wst, wells_closr, coll_pipe, \
plant, ORC_fluid, ORC_fluid_wst, diesel_stim, co2,_, _ = lookup_geothermal()
cooling_tower=bw.Database("geothermal energy").search("cooling tower")[0].key

list_act = [wellhead, diesel, steel, cement, water, 
         drilling_mud, drill_wst, wells_closr, coll_pipe,
         plant, cooling_tower, ORC_fluid, ORC_fluid_wst, diesel_stim]

# Retrieve methods 
ILCD_CC = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "climate change total" in str(method)]
ILCD_HH = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "human health" in str(method)]
ILCD_EQ = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "ecosystem quality" in str(method)]
ILCD_RE = [method for method in bw.methods if "ILCD 2.0 2018 midpoint no LT" in str(method) and "resources" in str(method)]
ILCD = ILCD_CC + ILCD_HH + ILCD_EQ + ILCD_RE

# Calculate impact of activities
lca = bw.LCA({list_act[0]: 1}, ILCD[0])
lca.lci()
lca.lcia()
coeff = {}
for method in ILCD:
    s=[]
    lca.switch_method(method)
    for act in list_act:
        lca.redo_lcia({act: 1})
        s.append(lca.score)
    coeff[method] = s

# Retrieve CF for co2 emissions    
for method in ILCD:
    CFs = bw.Method(method).load()
    coeff[method].append(next((cf[1] for cf in CFs if cf[0]==co2),0))

# Build matrix
col_names = ["wellhead", "diesel", "steel", "cement", "water", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid", "ORC_fluid_wst", "diesel_stim", "co2"]
coeff_matrix = pd.DataFrame.from_dict(coeff, orient="index", columns=col_names)

#Constant
drilling_waste_per_metre = 450

# Re-arrange matrix
coeff_matrix["concrete"] = coeff_matrix["cement"] + coeff_matrix["water"] * 1/0.65
coeff_matrix["ORC_fluid_tot"] = coeff_matrix["ORC_fluid"] - coeff_matrix["ORC_fluid_wst"]
coeff_matrix["electricity_stim"] = coeff_matrix["diesel_stim"] * 3.6 / 0.3
coeff_matrix["drill_wst"] = coeff_matrix["drill_wst"] * -1 * drilling_waste_per_metre
coeff_matrix=coeff_matrix.drop(columns=["cement", "ORC_fluid", "ORC_fluid_wst", "diesel_stim"])

col_ord = ["wellhead", "diesel", "steel", "concrete", \
         "drilling_mud", "drill_wst", "wells_closr", "coll_pipe", \
         "plant", "cooling tower", "ORC_fluid_tot", "water", "electricity_stim", "co2"]
coeff_matrix=coeff_matrix[col_ord]

is_= ["i1", "i2.1", "i2.2", "i2.3", "i2.4", "i2.5", \
      "i2.6","i3", "i4.1", "i4.2", "i4.3", "i5.1", "i5.2", "i6"]

if len(coeff_matrix.columns) == len(is_):
    coeff_matrix.columns=is_

In [2]:
coeff_matrix


,i1,i2.1,i2.2,i2.3,i2.4,i2.5,i2.6,i3,i4.1,i4.2,i4.3,i5.1,i5.2,i6
"(ILCD 2.0 2018 midpoint no LT, climate change, climate change total)",41951.729340,8.777813e-02,1.924688e+00,8.724747e-01,4.170591e+01,3.249762e+00,1.619566e+00,618.833223,1.363071e+05,2.901737e+05,1.933752e+01,1.099648e-03,3.110213e+00,1.0
"(ILCD 2.0 2018 midpoint no LT, human health, carcinogenic effects)",0.009333,1.338255e-10,6.007013e-07,3.485216e-09,5.328093e-07,4.229569e-05,9.236655e-09,0.000125,1.674126e-02,1.206404e-02,2.081519e-07,8.015878e-11,1.397821e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ionising radiation)",1130.168402,5.180734e-03,5.165839e-02,1.276488e-02,8.931351e-01,2.887734e-01,3.551923e-02,15.724582,4.156112e+03,2.173757e+04,2.324622e-01,7.549285e-05,1.798956e-01,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, non-carcinogenic effects)",0.022149,1.937160e-09,1.379242e-06,4.700238e-08,5.492678e-06,1.311540e-04,1.095188e-07,0.000300,6.103437e-02,1.316675e-01,1.493808e-06,3.343961e-10,1.017966e-07,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, ozone layer depletion)",0.002771,1.917973e-08,1.135916e-07,3.159978e-08,4.214417e-06,1.003866e-06,8.984558e-08,0.000039,1.270293e-02,1.479841e-01,7.843954e-04,7.157074e-11,6.616887e-07,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, photochemical ozone creation)",198.841132,1.590873e-03,9.231551e-03,1.970287e-03,1.899794e-01,2.701972e-02,4.846082e-03,2.818856,5.916199e+02,8.933771e+02,4.296965e-02,3.300982e-06,5.476726e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, human health, respiratory effects, inorganics)",0.003734,1.572051e-09,1.752617e-07,1.961009e-08,2.025714e-06,3.563340e-07,5.230986e-08,0.000051,9.882556e-03,1.355099e-02,6.220811e-07,5.597777e-11,6.149157e-08,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater and terrestrial acidification)",225.039103,1.242693e-03,9.812659e-03,2.418912e-03,1.981686e-01,2.520450e-02,5.682148e-03,3.459902,9.570330e+02,1.766334e+03,6.213826e-02,5.628260e-06,4.321668e-02,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater ecotoxicity)",118226.208018,1.458437e-02,7.242976e+00,1.629244e-01,2.966899e+01,2.357621e+03,6.021999e-01,1587.931584,2.904214e+05,3.394154e+05,8.931917e+00,1.298698e-03,6.767038e-01,0.0
"(ILCD 2.0 2018 midpoint no LT, ecosystem quality, freshwater eutrophication)",1.973278,1.373653e-07,1.131584e-04,1.085733e-05,2.125317e-03,6.831965e-03,2.468011e-05,0.028690,7.573346e+00,3.852353e+01,2.961919e-04,6.684274e-08,1.081121e-05,0.0


# Enhanced geothermal

# Define symbolic

In [3]:
from sympy import symbols, collect, ratsimp, fraction, factor_terms

In [4]:
# Coefficients
i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6 = \
symbols('i1, i2_1, i2_2, i2_3, i2_4, i2_5, i2_6, i3, i4, i4_1, i4_2, i4_3, i5_1, i5_2, i6')

# Main parameters
W_Pn, P_ne , PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, S_w, S_el  = \
symbols('W_Pn, P_ne, PIratio, W_d, E_co2, SR_p, D_i, LT, SR_m, SR_e, D, C_S, C_C, DM, CP, CF, AP, SW_n, SW, S_el')

# Other variables that are actually irrelevant, because they will be multiplied by 0 in the enhanced model
# I wanna keep them though for the sake of completeness
E_co2 = symbols('E_co2')

# Constants
CT_el, CT_n, W_en, OF = symbols ('CT_el, CT_n, W_E_en, OF')


# Main equation


In [5]:
# Number of wells with success rate
W_n = W_Pn/SR_p
W_E_en = W_en/SR_e

In [6]:
# Equation
nominator = (W_n + W_E_en) *i1 \
          + W_d* (W_n + W_E_en) * (D*i2_1 + C_S*i2_2 + C_C*i2_3  +  DM*i2_4 + i2_5 + i2_6) + \
          + W_n*CP*i3 \
          + P_ne*i4 \
          + SW_n*S_w * (i5_1 + S_el*i5_2)

c1 = P_ne*CF*(1-AP)*LT*8760000
d1 = CT_el*CT_n*1000*LT
denominator = c1  -  d1

summand = E_co2*i6

eq = nominator/denominator + summand
eq = eq.subs(i4, i4_1 + i4_2*CT_n + i4_3*OF)

In [7]:
eq

E_co2*i6 + (CP*W_Pn*i3/SR_p + P_ne*(CT_n*i4_2 + OF*i4_3 + i4_1) + SW*SW_n*(S_el*i5_2 + i5_1) + W_d*(W_Pn/SR_p + W_E_en/SR_e)*(C_C*i2_3 + C_S*i2_2 + D*i2_1 + DM*i2_4 + i2_5 + i2_6) + i1*(W_Pn/SR_p + W_E_en/SR_e))/(8760000*CF*LT*P_ne*(1 - AP) - 1000*CT_el*CT_n*LT)

# Get parameters value

In [8]:
# Klausen
from ege_klausen import parameters
parameters.static()   

# Parameters of simplified models
D_val      = parameters["specific_diesel_consumption"]
W_d_val      = parameters["average_depth_of_wells"]
P_ne_val     = parameters["installed_capacity"]
SR_p_val     = parameters["success_rate_primary_wells"]/100

# Parameters to be fixed
LT_val     = parameters["lifetime"]
SR_e_val   = parameters["success_rate_exploration_wells"] / 100
C_S_val    = parameters["specific_steel_consumption"]
C_C_val    = parameters["specific_cement_consumption"]
DM_val     = parameters["specific_drilling_mud_consumption"]
CP_val     = parameters["collection_pipelines"]
CF_val     = parameters["capacity_factor"]
AP_val    = parameters["auxiliary_power"]
W_Pn_val  = parameters["number_of_wells"] 
SW_n_val   = np.round(0.5 + parameters["number_of_wells_stimulated_0to1"] * parameters["number_of_wells"])
S_w_val    = parameters["water_stimulation"]
S_el_val   = parameters["specific_electricity_stimulation"] / 1000

# Parameters that are not used in ENHANCED
D_i_val     = 0
E_co2_val   = 0

# Constants
CT_el_val  = 864
CT_n_val   = 7/303.3
W_en_val   = 3 * 0.3
OF_val         = 300

par_dict = { LT: LT_val,
             SR_e: SR_e_val,
             C_S: C_S_val,
             C_C: C_C_val,
             DM: DM_val,
             CP: CP_val,
             CF: CF_val,
             AP: AP_val,
             W_Pn: W_Pn_val,
             SW_n: SW_n_val,
             S_w: S_w_val,
             S_el: S_el_val,
             D_i: D_i_val,
             E_co2: E_co2_val, 
             CT_el: CT_el_val,
             CT_n: CT_n_val,
             W_en: W_en_val,
             OF: OF_val}

# Groups of environmental categories

In [9]:
# Group2 uses Diesel consumption


group1_05 = ["carcinogenic effects", "non-carcinogenic effects", "respiratory effects, inorganics",  "freshwater ecotoxicity",
             "freshwater eutrophication", "minerals and metals", "dissipated water", "land use"]  

group2_05 = ["climate change total", "ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication", "fossils"]


group1_10 = ["climate change total",  "carcinogenic effects", "non-carcinogenic effects",\
              "respiratory effects, inorganics",  "freshwater ecotoxicity", "freshwater eutrophication", \
              "minerals and metals", "dissipated water", "fossils", "land use"]            

group2_10 = ["ionising radiation", "ozone layer depletion", "photochemical ozone creation", \
            "freshwater and terrestrial acidification", "marine eutrophication", "terrestrial eutrophication"]


group1_15 = ["climate change total", "carcinogenic effects", "ionising radiation", "non-carcinogenic effects", \
            "ozone layer depletion", "photochemical ozone creation", "respiratory effects, inorganics", \
            "freshwater and terrestrial acidification", "freshwater ecotoxicity", "freshwater eutrophication", \
            "minerals and metals", "dissipated water", "fossils", "land use"]

group2_15 = ["marine eutrophication", "terrestrial eutrophication"]

group_20 = [method[2] for method in ILCD]


# Simplified equations

In [10]:
chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, delta_6,\
delta_7, delta_8, delta_9 = \
symbols('chi_1, chi_2, chi_3, chi_4, chi_5, chi_6, chi_7, delta_1, delta_2, delta_3, delta_4, delta_5, \
delta_6, delta_7, delta_8, delta_9')


## Threshold 20% - one group only

In [11]:
eq_group_20 = chi_1/(chi_2*P_ne - chi_3) + chi_4#
eq_group_20

chi_1/(P_ne*chi_2 - chi_3) + chi_4

## Threhold 15/10% - group 1

In [12]:
eq_group1_15_10 = chi_1/(chi_2*P_ne - chi_3) + chi_4
eq_group1_15_10

chi_1/(P_ne*chi_2 - chi_3) + chi_4

## Threhold 15/10% - group 2

In [13]:
eq_group2_15_10 = (D*delta_1 + delta_2) / (P_ne*delta_3 - delta_4) + delta_5
eq_group2_15_10

delta_5 + (D*delta_1 + delta_2)/(P_ne*delta_3 - delta_4)

## Threshold 5% - group1

In [14]:
eq_group1_05 = (SR_p*W_d*chi_1 + SR_p*chi_2 + W_d*chi_3 + chi_4) / (P_ne*SR_p*chi_5 - SR_p*chi_6) +chi_7
eq_group1_05

chi_7 + (SR_p*W_d*chi_1 + SR_p*chi_2 + W_d*chi_3 + chi_4)/(P_ne*SR_p*chi_5 - SR_p*chi_6)

## Threshold 5% - group2

In [15]:
eq_group2_05 = (D*SR_p*W_d*delta_1 + D*W_d*delta_2 + SR_p*W_d*delta_3 + SR_p*delta_4 + W_d*delta_5 + delta_6 ) \
/ (P_ne*SR_p*delta_7 - SR_p*delta_8) + delta_9
eq_group2_05

delta_9 + (D*SR_p*W_d*delta_1 + D*W_d*delta_2 + SR_p*W_d*delta_3 + SR_p*delta_4 + W_d*delta_5 + delta_6)/(P_ne*SR_p*delta_7 - SR_p*delta_8)

# Compute chi

## Threshold = 20%

In [16]:
# Replace parameters in equation
eq_chi_20 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        D: D_val,
        W_d: W_d_val}
eq_chi_20 = eq_chi_20.subs(repl)

chi_dict_20 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group_20:
        eq_chi_20_is = eq_chi_20.subs(is_dict)
        eq_chi_20_is = ratsimp(eq_chi_20_is)
              
        chi4 = eq_chi_20_is.args[0] 
        chi1 = fraction(eq_chi_20_is.args[1])[0]
        
        den = fraction(eq_chi_20_is.args[1])[1]
        chi2 = collect(den, P_ne, evaluate=False)[P_ne]
        chi3 = collect(den, P_ne, evaluate=False)[1]
        
        chi_dict_20[str(method)] = {'chi1': chi1, 'chi2': chi2,
                                   'chi3': chi3, 'chi4': chi4}
           

In [17]:
# Show example of the formula
eq_chi_20_is

4.16165746936438e-8 + 345.157897572304/(189216000.0*P_ne - 598219.584569733)

In [18]:
chi_df_20 = pd.DataFrame.from_dict(chi_dict_20, orient='index')
chi_df_20 = chi_df_20.astype(float)
chi_df_20

,chi1,chi2,chi3,chi4
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",4.138549e+07,189216000.0,-598219.58457,7.864316e-04
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",2.802742e+00,189216000.0,-598219.58457,9.027851e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",2.167540e+06,189216000.0,-598219.58457,2.498490e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",8.061551e+00,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",7.756174e+00,189216000.0,-598219.58457,1.328836e-09
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",6.369133e+05,189216000.0,-598219.58457,3.303788e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",1.213665e+00,189216000.0,-598219.58457,5.486814e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",5.099870e+05,189216000.0,-598219.58457,5.371853e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",8.284368e+07,189216000.0,-598219.58457,1.590428e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",6.192698e+02,189216000.0,-598219.58457,4.519336e-08


## Threshold 15%

In [19]:
# Select from chi_df_20 the categories that are in group1_15
mask =[chi_df_20.index.str.contains(method) for method in group1_15]
chi_df_15 = pd.concat([chi_df_20[m] for m in mask])
chi_df_15

,chi1,chi2,chi3,chi4
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",4.138549e+07,189216000.0,-598219.58457,7.864316e-04
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",2.802742e+00,189216000.0,-598219.58457,9.027851e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",8.061551e+00,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",2.167540e+06,189216000.0,-598219.58457,2.498490e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",8.061551e+00,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",7.756174e+00,189216000.0,-598219.58457,1.328836e-09
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",6.369133e+05,189216000.0,-598219.58457,3.303788e-06
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",1.213665e+00,189216000.0,-598219.58457,5.486814e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",5.099870e+05,189216000.0,-598219.58457,5.371853e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",8.284368e+07,189216000.0,-598219.58457,1.590428e-03


## Threshold 10%

In [20]:
# Select from chi_df_20 the categories that are in group1_15
mask =[chi_df_20.index.str.contains(method) for method in group1_10]
chi_df_10 = pd.concat([chi_df_20[m] for m in mask])
chi_df_10

,chi1,chi2,chi3,chi4
"('ILCD 2.0 2018 midpoint no LT', 'climate change', 'climate change total')",4.138549e+07,189216000.0,-598219.58457,7.864316e-04
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",2.802742e+00,189216000.0,-598219.58457,9.027851e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",8.061551e+00,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",8.061551e+00,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",1.213665e+00,189216000.0,-598219.58457,5.486814e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",8.284368e+07,189216000.0,-598219.58457,1.590428e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",6.192698e+02,189216000.0,-598219.58457,4.519336e-08
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')",3.451579e+02,189216000.0,-598219.58457,4.161657e-08
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water')",4.130554e+06,189216000.0,-598219.58457,3.853415e-04
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'fossils')",5.550806e+08,189216000.0,-598219.58457,1.065016e-02


## Threshold 5%

In [21]:
# Replace parameters in equation
eq_chi_05 = eq.subs(par_dict)
repl = {D: D_val}
eq_chi_05 = eq_chi_05.subs(repl)

chi_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_chi_05_is = eq_chi_05.subs(is_dict)
        eq_chi_05_is = ratsimp(eq_chi_05_is)
              
        chi7 = eq_chi_05_is.args[0] 
        num =fraction(eq_chi_05_is.args[1])[0]
        chi1 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        chi2 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        chi3 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
        chi4 = collect(num, [SR_p*W_d, SR_p, W_d], evaluate=False)[1]
        
        den = fraction(eq_chi_05_is.args[1])[1]
        chi5 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[P_ne*SR_p]
        chi6 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[SR_p]
        
        chi_dict_05[str(method)] = {'chi1': chi1, 'chi2': chi2,
                                    'chi3': chi3, 'chi4': chi4,
                                    'chi5': chi5, 'chi6': chi6,
                                    'chi7': chi7}
           

In [22]:
# Show example of the formula
eq_chi_05_is

4.16165746936438e-8 + 1.0*(0.0128092044494421*SR_p*W_d + 95.9305244339533*SR_p + 0.0286820282681822*W_d + 18.6048676143821)/(189216000.0*P_ne*SR_p - 598219.584569733*SR_p)

In [23]:
chi_df_05 = pd.DataFrame.from_dict(chi_dict_05, orient='index')
chi_df_05 = chi_df_05.astype(float)
chi_df_05

,chi1,chi2,chi3,chi4,chi5,chi6,chi7
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",0.000149,9.155427e-02,0.000335,7.486187e-02,189216000.0,-598219.58457,9.027851e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",0.000414,5.877192e-01,0.000928,1.788901e-01,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",0.000048,3.317846e-01,0.000108,3.022895e-02,189216000.0,-598219.58457,5.486814e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",4455.787171,3.802952e+06,9977.279549,9.501530e+05,189216000.0,-598219.58457,1.590428e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",0.030501,6.410861e+01,0.068297,1.667844e+01,189216000.0,-598219.58457,4.519336e-08
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water')",194.621908,5.431820e+05,435.792175,1.388747e+05,189216000.0,-598219.58457,3.853415e-04
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'land use')",2972.949305,1.245212e+07,6656.948631,2.365012e+06,189216000.0,-598219.58457,5.907572e-03
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')",0.012809,9.593052e+01,0.028682,1.860487e+01,189216000.0,-598219.58457,4.161657e-08


# Compute delta

## Threshold = 15%

In [24]:
# Replace parameters in equation
eq_delta_15 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_15 = eq_delta_15.subs(repl)

delta_dict_15 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_15:
        eq_delta_15_is = eq_delta_15.subs(is_dict)
        eq_delta_15_is = ratsimp(eq_delta_15_is)
              
        delta5 = eq_delta_15_is.args[0] 
        
        num  = fraction(eq_delta_15_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
        
        den  = fraction(eq_delta_15_is.args[1])[1]
        delta3 = collect(den, [P_ne], evaluate=False)[P_ne]
        delta4 = collect(den, [P_ne], evaluate=False)[1]
                
        delta_dict_15[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5}

In [25]:
# Show example of the formula
eq_delta_15_is

1.0*(142.061722936549*D + 1182239.93845664)/(189216000.0*P_ne - 598219.584569733) + 9.59210575558912e-6

In [26]:
delta_df_15 = pd.DataFrame.from_dict(delta_dict_15, orient='index')
delta_df_15 = delta_df_15.astype(float)
delta_df_15

,delta1,delta2,delta3,delta4,delta5
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",12.967920,1.082171e+05,189216000.0,-598219.58457,0.000001
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'terrestrial eutrophication')",142.061723,1.182240e+06,189216000.0,-598219.58457,0.000010


## Threshold = 10%

In [27]:
# Replace parameters in equation
eq_delta_10 = eq.subs(par_dict)
repl = {SR_p: SR_p_val,
        W_d: W_d_val}
eq_delta_10 = eq_delta_10.subs(repl)

delta_dict_10 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group2_10:
        eq_delta_10_is = eq_delta_10.subs(is_dict)
        eq_delta_10_is = ratsimp(eq_delta_10_is)
              
        delta5 = eq_delta_10_is.args[0] 
        
        num  = fraction(eq_delta_10_is.args[1])[0]
        delta1 = collect(num, [D], evaluate=False)[D]
        delta2 = collect(num, [D], evaluate=False)[1]
        
        den  = fraction(eq_delta_10_is.args[1])[1]
        delta3 = collect(den, [P_ne], evaluate=False)[P_ne]
        delta4 = collect(den, [P_ne], evaluate=False)[1]
                
        delta_dict_10[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5}

In [28]:
# Show example of the formula
eq_delta_15_is

1.0*(142.061722936549*D + 1182239.93845664)/(189216000.0*P_ne - 598219.584569733) + 9.59210575558912e-6

In [29]:
delta_df_10 = pd.DataFrame.from_dict(delta_dict_10, orient='index')
delta_df_10 = delta_df_10.astype(float)
delta_df_10

,delta1,delta2,delta3,delta4,delta5
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ionising radiation')",121.074485,1.138407e+06,189216000.0,-598219.58457,2.498490e-05
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'ozone layer depletion')",0.000448,3.946193e+00,189216000.0,-598219.58457,1.328836e-09
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'photochemical ozone creation')",37.178929,3.208924e+05,189216000.0,-598219.58457,3.303788e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater and terrestrial acidification')",29.041912,2.631307e+05,189216000.0,-598219.58457,5.371853e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'marine eutrophication')",12.967920,1.082171e+05,189216000.0,-598219.58457,1.008642e-06
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'terrestrial eutrophication')",142.061723,1.182240e+06,189216000.0,-598219.58457,9.592106e-06


## Threshold = 5%

In [30]:
# Replace parameters in equation
eq_delta_05 = eq.subs(par_dict)

delta_dict_05 = {}
for method in ILCD:
    is_ = dict(coeff_matrix.T[method])
    is_dict = {k.replace('.', '_'): v for k,v in is_.items()}
    if method[2] in group1_05:
        eq_delta_05_is = eq_delta_05.subs(is_dict)
        eq_delta_05_is = ratsimp(eq_delta_05_is)
              
        delta9 = eq_delta_05_is.args[0] 
        
        num = fraction(eq_delta_05_is.args[1])[0]
        delta1 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*SR_p*W_d]
        delta2 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[D*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p*W_d]
        delta3 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[SR_p]
        delta5 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[W_d]
        delta6 = collect(num, [D*SR_p*W_d, D*W_d, SR_p*W_d, SR_p, W_d], evaluate=False)[1]
        
        den = fraction(eq_delta_05_is.args[1])[1]
        delta7 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[P_ne*SR_p]
        delta8 = collect(den, [P_ne*SR_p, SR_p], evaluate=False)[SR_p]
        
        delta_dict_05[str(method)] = {'delta1': delta1, 'delta2': delta2,
                                    'delta3': delta3, 'delta4': delta4,
                                    'delta5': delta5, 'delta6': delta6,
                                    'delta7': delta7, 'delta8': delta8,
                                    'delta9': delta9}

In [31]:
# Show example of the formula
eq_delta_05_is

4.16165746936438e-8 + 1.0*(1.08183696540838e-7*D*SR_p*W_d + 2.42242042008777e-7*D*W_d + 0.0118896430288449*SR_p*W_d + 95.9305244339533*SR_p + 0.0266229709111076*W_d + 18.6048676143821)/(189216000.0*P_ne*SR_p - 598219.584569733*SR_p)

In [32]:
delta_df_05 = pd.DataFrame.from_dict(delta_dict_05, orient='index')
delta_df_05 = delta_df_05.astype(float)
delta_df_05

,delta1,delta2,delta3,delta4,delta5,delta6,delta7,delta8,delta9
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'carcinogenic effects')",1.792967e-10,4.014764e-10,9.155427e-02,-598219.58457,0.000331,7.486187e-02,189216000.0,-598219.58457,9.027851e-11
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'non-carcinogenic effects')",2.595368e-09,5.811479e-09,5.877192e-01,-598219.58457,0.000878,1.788901e-01,189216000.0,-598219.58457,3.409930e-10
"('ILCD 2.0 2018 midpoint no LT', 'human health', 'respiratory effects, inorganics')",2.106203e-09,4.716153e-09,3.317846e-01,-598219.58457,0.000068,3.022895e-02,189216000.0,-598219.58457,5.486814e-11
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater ecotoxicity')",1.953985e-02,4.375312e-02,3.802952e+06,-598219.58457,9605.378061,9.501530e+05,189216000.0,-598219.58457,1.590428e-03
"('ILCD 2.0 2018 midpoint no LT', 'ecosystem quality', 'freshwater eutrophication')",1.840393e-07,4.120958e-07,6.410861e+01,-598219.58457,0.064794,1.667844e+01,189216000.0,-598219.58457,4.519336e-08
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'dissipated water')",2.107576e-03,4.719229e-03,5.431820e+05,-598219.58457,395.678729,1.388747e+05,189216000.0,-598219.58457,3.853415e-04
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'land use')",6.578688e-02,1.473082e-01,1.245212e+07,-598219.58457,5404.828783,2.365012e+06,189216000.0,-598219.58457,5.907572e-03
"('ILCD 2.0 2018 midpoint no LT', 'resources', 'minerals and metals')",1.081837e-07,2.422420e-07,9.593052e+01,-598219.58457,0.026623,1.860487e+01,189216000.0,-598219.58457,4.161657e-08


# Save to excel

In [33]:
with pd.ExcelWriter(os.path.join(path, 'generated_files\Simplified models coefficients enhanced - symbolic - thresholds.xlsx')) as writer:
    chi_df_05.to_excel(writer, sheet_name='chi_5%')
    chi_df_10.to_excel(writer, sheet_name='chi_10%')
    chi_df_15.to_excel(writer, sheet_name='chi_15%')
    chi_df_20.to_excel(writer, sheet_name='chi_20%')    
    delta_df_05.to_excel(writer, sheet_name='delta_5%')
    delta_df_10.to_excel(writer, sheet_name='delta_10%')
    delta_df_15.to_excel(writer, sheet_name='delta_15%')
    